In [ ]:
!pip install ultralytics
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 37.6 MB/s eta 0:00:00


In [ ]:
# Скачиваем нужный датасет
!kaggle datasets download -d vinayakshanawad/meat-freshness-image-dataset

Dataset URL: https://www.kaggle.com/datasets/vinayakshanawad/meat-freshness-image-dataset
License(s): CC0-1.0
 91% 54.0M/59.4M [00:02<00:00, 31.6MB/s]
100% 59.4M/59.4M [00:02<00:00, 23.8MB/s]


In [ ]:
# Распаковка
!unzip meat-freshness-image-dataset.zip

Archive:  meat-freshness-image-dataset.zip
  inflating: Meat Freshness.v1-new-dataset.multiclass/README.dataset.txt  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-100-_JPG.rf.65e835ab6a2890785aade192d1e19549.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1000-_JPG.rf.d0f18a5c4075b50beeb7ee99036d491d.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1003-_JPG.rf.b410c33a04a1fba90032a9086d191d6d.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1005-_JPG.rf.404839308dfdce0228de77bf7458513a.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1009-_JPG.rf.1c4c304a072d95160d6ec26f0191344e.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/FRESH-101-_JPG.rf.4c18cd9fd197ac72fa9dd68fcd00502f.jpg  
  inflating: Meat Freshness.v1-new-dataset.multiclass/train/F

In [ ]:
import os
import shutil

# Указываем пути к папкам train и valid
base_dir = "/content/Meat Freshness.v1-new-dataset.multiclass"  # Замените на путь к вашей папке
folders = ["train", "valid"]

for folder in folders:
    folder_path = os.path.join(base_dir, folder)

    # Создаём папки для классов, если их ещё нет
    for class_name in ["FRESH", "HALF", "SPOILED"]:
        os.makedirs(os.path.join(folder_path, class_name), exist_ok=True)

    # Перебираем файлы в папке
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Пропускаем файлы CSV или другие неподходящие форматы
        if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        # Переименовываем файлы с "HALF-FRESH" -> "HALF"
        if file_name.startswith("HALF-FRESH"):
            new_name = file_name.replace("HALF-FRESH", "HALF", 1)
            new_file_path = os.path.join(folder_path, new_name)
            os.rename(file_path, new_file_path)
            file_path = new_file_path  # Обновляем путь файла после переименования

        # Определяем класс по префиксу имени файла
        if file_name.startswith("FRESH"):
            class_folder = "FRESH"
        elif file_name.startswith("HALF"):
            class_folder = "HALF"
        elif file_name.startswith("SPOILED"):
            class_folder = "SPOILED"
        else:
            continue  # Пропускаем файлы без соответствующего префикса

        # Перемещаем файл в соответствующую папку
        shutil.move(file_path, os.path.join(folder_path, class_folder, file_name))


In [ ]:
from ultralytics import YOLO
# Загрузите YOLOv11 модель для классификации
model = YOLO('yolov8x-cls.pt')  # Выберите базовую модель; можно взять и другие модели (s, m, l, x)

100%|██████████| 110M/110M [00:10<00:00, 10.7MB/s]


In [ ]:
# Обучение
model.train(data=r'/content/Meat Freshness.v1-new-dataset.multiclass', epochs=10, imgsz=256, batch=16)

Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/content/Meat Freshness.v1-new-dataset.multiclass, epochs=10, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

train: Scanning /content/Meat Freshness.v1-new-dataset.multiclass/train... 1815 images, 0 corrupt: 100%|██████████| 1815/1815 [00:00<?, ?it/s]
val: Scanning /content/Meat Freshness.v1-new-dataset.multiclass/val... 451 images, 0 corrupt: 100%|██████████| 451/451 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/classify/train5
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 15/15 [00:02<00:00,  7.36it/s]

                   all      0.987          1


...

                   all      0.996          1



10 epochs completed in 0.088 hours.
Optimizer stripped from runs/classify/train5/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train5/weights/best.pt, 112.5MB

Validating runs/classify/train5/weights/best.pt...
Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,127,043 parameters, 0 gradients, 153.8 GFLOPs
train: /content/Meat Freshness.v1-new-dataset.multiclass/train... found 1815 images in 3 classes ✅ 
val: /content/Meat Freshness.v1-new-dataset.multiclass/val... found 451 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 15/15 [00:02<00:00,  7.07it/s]


                   all      0.996          1
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6221da26e0>
curves: []
curves_results: []
fitness: 0.9977827072143555
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9955654144287109, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9977827072143555}
save_dir: PosixPath('runs/classify/train5')
speed: {'preprocess': 0.13671058773201478, 'inference': 2.6189614822489196, 'loss': 0.0007697061001593681, 'postprocess': 0.0011566737274372922}
task: 'classify'
top1: 0.9955654144287109
top5: 1.0

In [ ]:
import matplotlib.pyplot as plt

# Загружаем модель
model = YOLO('/content/runs/classify/train3/weights/best.pt')
img = '/content/Meat Freshness.v1-new-dataset.multiclass/test/SPOILED/SPOILED-726-_JPG.rf.84dacc087202efc5f2c8cd50b7d47974.jpg'
results = model(img)


image 1/1 /content/Meat Freshness.v1-new-dataset.multiclass/test/SPOILED/SPOILED-726-_JPG.rf.84dacc087202efc5f2c8cd50b7d47974.jpg: 256x256 SPOILED 1.00, FRESH 0.00, HALF 0.00, 4.0ms
Speed: 6.3ms preprocess, 4.0ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)


In [ ]:
features = model(img)
print(features)


0: 256x256 SPOILED 1.00, FRESH 0.00, HALF 0.00, 8.1ms
Speed: 8.1ms preprocess, 8.1ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)
[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'FRESH', 1: 'HALF', 2: 'SPOILED'}
obb: None
orig_img: array([[[ 25,  36,  63],
        [ 25,  36,  63],
        [ 25,  36,  63],
        ...,
        [ 26,  44,  75],
        [ 25,  41,  70],
        [ 22,  38,  67]],

       [[ 24,  35,  62],
        [ 24,  35,  62],
        [ 23,  34,  61],
        ...,
        [ 28,  46,  77],
        [ 28,  44,  73],
        [ 25,  41,  70]],

       [[ 22,  34,  62],
        [ 22,  34,  62],
        [ 21,  33,  61],
        ...,
        [ 30,  48,  79],
        [ 31,  47,  76],
        [ 30,  46,  75]],

       ...,

       [[181, 196, 188],
        [178, 194, 183],
        [179, 194, 186],
        ...,
        [ 28,  27,  31],
        [ 21,  20,  24],
        [ 23,  22,  26]],

       [[1

In [ ]:
# Выполняем классификацию с указанием save_dir
features = model('/content/Meat Freshness.v1-new-dataset.multiclass/test/SPOILED/SPOILED-726-_JPG.rf.84dacc087202efc5f2c8cd50b7d47974.jpg', save=True)



image 1/1 /content/Meat Freshness.v1-new-dataset.multiclass/test/SPOILED/SPOILED-726-_JPG.rf.84dacc087202efc5f2c8cd50b7d47974.jpg: 256x256 SPOILED 1.00, FRESH 0.00, HALF 0.00, 7.3ms
Speed: 6.9ms preprocess, 7.3ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)
Results saved to runs/classify/predict


In [ ]:
import shutil
from google.colab import files


# Укажите путь к папке, которую хотите архивировать
folder_path = '/content/runs/classify/train5'

# Укажите имя архива и его расположение
output_filename = '/content/model_x.zip'

# Архивируем папку
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_path)

print(f"Папка упакована в архив: {output_filename}")

# Укажите путь к архиву
#files.download('/content/model_n.zip')

Папка упакована в архив: /content/model_x.zip
